In [26]:
import numpy as np
import pickle
import os

# Function to unpickle CIFAR-10 files
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

# Define the path where your CIFAR-10 files are stored
data_path = '/kaggle/input/cifar10/cifar-10-batches-py'  # Adjust if necessary

# Load all training batches
def load_cifar10(data_path):
    train_data = []
    train_labels = []
    
    # Load all batch files
    for i in range(1, 6):
        batch_file = os.path.join(data_path, f'data_batch_{i}')
        batch = unpickle(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    # Combine all batches
    train_data = np.vstack(train_data).reshape(-1, 3, 32, 32).astype("float32")
    train_labels = np.array(train_labels)

    # Load test batch
    test_batch = unpickle(os.path.join(data_path, 'test_batch'))
    test_data = test_batch[b'data'].reshape(-1, 3, 32, 32).astype("float32")
    test_labels = np.array(test_batch[b'labels'])

    # Load label names
    meta = unpickle(os.path.join(data_path, 'batches.meta'))
    label_names = meta[b'label_names']
    label_names = [label.decode('utf-8') for label in label_names]

    return train_data, train_labels, test_data, test_labels, label_names

# Load the CIFAR-10 data
train_data, train_labels, test_data, test_labels, label_names = load_cifar10(data_path)

# Verify the data
print(f"Training Data Shape: {train_data.shape}")
print(f"Training Labels Shape: {train_labels.shape}")
print(f"Test Data Shape: {test_data.shape}")
print(f"Test Labels Shape: {test_labels.shape}")
print(f"Label Names: {label_names}")


Training Data Shape: (50000, 3, 32, 32)
Training Labels Shape: (50000,)
Test Data Shape: (10000, 3, 32, 32)
Test Labels Shape: (10000,)
Label Names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [27]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# Normalize the data and convert to tensors
from torchvision.transforms import ToPILImage

# Define data augmentation for training
transform_train = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # CIFAR-10 mean and std
])

# Define transformations for testing
transform_test = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


# Custom PyTorch Dataset class
class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].transpose(1, 2, 0)  # Convert to HWC format for transformations
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# Create datasets
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=transform_train)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=transform_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt

In [29]:
class FineTunedResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(FineTunedResNet, self).__init__()
        
        # Load ResNet-18 Pretrained Model
        self.resnet = models.resnet18(pretrained=True)
        
        # Modify first conv layer (ResNet expects 224x224 images, but CIFAR-10 is 32x32)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        
        # Freeze ResNet layers except the final FC layer
        for param in self.resnet.parameters():
            param.requires_grad = False  # Freeze
        
        # Replace classifier with LeakyCNN's fully connected layers
        self.resnet.fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.resnet(x)






In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models  # Import this line to access ResNet
import time

# Initialize the model (fine-tuned ResNet-18)
model = FineTunedResNet(num_classes=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Step 1: Train only the FC layer first
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=0.001)

def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagate the loss
        
        optimizer.step()  # Update parameters

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy

# Step 2: Fine-tune the entire model (unfreeze ResNet)
def unfreeze_and_finetune(model):
    # Unfreeze all layers in ResNet
    for param in model.resnet.parameters():
        param.requires_grad = True

    # Re-initialize the optimizer to include all parameters
    optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate for fine-tuning
    return optimizer

# Training function with two phases
def train_model(model, train_loader, val_loader, epochs=30):
    global optimizer  # Use the global optimizer

    best_acc = 0.0
    for epoch in range(epochs):
        start_time = time.time()

        # Train the model (Feature extraction phase)
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%')

        # After a few epochs, unfreeze ResNet and fine-tune (Full fine-tuning phase)
        if epoch == 5:  # Unfreeze after 5 epochs of training only the FC layer
            optimizer = unfreeze_and_finetune(model)

        # Optionally evaluate on validation set here

        print(f'Time taken for epoch {epoch + 1}: {time.time() - start_time:.2f} seconds')

# Train the model
train_model(model, train_loader, test_loader, epochs=30)


Epoch [1/30], Loss: 2.0373, Accuracy: 25.49%
Time taken for epoch 1: 16.38 seconds
Epoch [2/30], Loss: 1.9709, Accuracy: 28.34%
Time taken for epoch 2: 16.35 seconds
Epoch [3/30], Loss: 1.9504, Accuracy: 28.78%
Time taken for epoch 3: 15.92 seconds
Epoch [4/30], Loss: 1.9416, Accuracy: 29.12%
Time taken for epoch 4: 16.38 seconds
Epoch [5/30], Loss: 1.9364, Accuracy: 29.32%
Time taken for epoch 5: 16.42 seconds
Epoch [6/30], Loss: 1.9309, Accuracy: 29.90%
Time taken for epoch 6: 16.67 seconds
Epoch [7/30], Loss: 1.4155, Accuracy: 49.44%
Time taken for epoch 7: 25.05 seconds
Epoch [8/30], Loss: 1.0440, Accuracy: 63.85%
Time taken for epoch 8: 24.94 seconds
Epoch [9/30], Loss: 0.8717, Accuracy: 70.09%
Time taken for epoch 9: 24.82 seconds
Epoch [10/30], Loss: 0.7568, Accuracy: 74.16%
Time taken for epoch 10: 24.75 seconds
Epoch [11/30], Loss: 0.6698, Accuracy: 77.18%
Time taken for epoch 11: 24.78 seconds
Epoch [12/30], Loss: 0.6052, Accuracy: 79.23%
Time taken for epoch 12: 24.83 second